# 🚨 Emergency Vehicle Detection - YOLOv5 Training

**Dataset**: Emergency Vehicle Dataset (600 images, 6 classes)

**Model**: YOLOv5n (nano - fastest inference)

**Training Time**: ~20-30 minutes on T4 GPU

---

## 📋 Prerequisites
- ✅ Google Colab with GPU (Runtime > Change runtime type > T4 GPU)
- ✅ Dataset file: `dataset.zip` (download from Drive link in README_DATASET)

---

## 1️⃣ Setup Environment

In [ ]:
# Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ WARNING: No GPU detected! Training will be very slow.")
    print("   Go to: Runtime > Change runtime type > Hardware accelerator > GPU")

In [ ]:
# Clone YOLOv5 repository
import os

if not os.path.exists('yolov5'):
    print("📥 Cloning YOLOv5 repository...")
    !git clone https://github.com/ultralytics/yolov5
    print("✅ YOLOv5 cloned successfully")
else:
    print("✅ YOLOv5 already exists")

# Change to yolov5 directory
%cd yolov5

# Install dependencies
print("\n📦 Installing dependencies...")
!pip install -q -r requirements.txt
print("✅ Dependencies installed")

## 2️⃣ Upload & Prepare Dataset

**Option A**: Upload `dataset.zip` manually (click folder icon → upload)

**Option B**: Mount Google Drive and copy from there

In [ ]:
# OPTION A: Upload dataset.zip manually
# Uncomment below if you want to upload via code
# from google.colab import files
# print("📤 Please upload dataset.zip...")
# uploaded = files.upload()

# OPTION B: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Copy from Drive (adjust path to your file location)
import shutil
drive_path = '/content/drive/MyDrive/dataset.zip'  # ⚠️ ADJUST THIS PATH

if os.path.exists(drive_path):
    print(f"📋 Copying dataset from Drive...")
    shutil.copy(drive_path, '/content/yolov5/dataset.zip')
    print("✅ Dataset copied")
else:
    print(f"❌ File not found at: {drive_path}")
    print("   Please upload dataset.zip manually or adjust the path")

In [ ]:
# Extract and verify dataset
import zipfile

zip_path = '/content/yolov5/dataset.zip'
extract_path = '/content/yolov5/'

# Check if zip exists
if not os.path.exists(zip_path):
    raise FileNotFoundError(f"❌ dataset.zip not found at {zip_path}")

# Extract
print("📦 Extracting dataset...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print("✅ Dataset extracted")

# Verify structure
print("\n📁 Dataset structure:")
!ls -la /content/yolov5/dataset/

print("\n📊 Dataset statistics:")
train_images = len(os.listdir('/content/yolov5/dataset/train/images'))
val_images = len(os.listdir('/content/yolov5/dataset/valid/images'))
print(f"  Training images: {train_images}")
print(f"  Validation images: {val_images}")
print(f"  Total: {train_images + val_images}")

# Verify data.yaml
yaml_path = '/content/yolov5/dataset/data.yaml'
if os.path.exists(yaml_path):
    print(f"\n✅ data.yaml found")
    print("\n📄 data.yaml content:")
    !cat /content/yolov5/dataset/data.yaml
else:
    raise FileNotFoundError(f"❌ data.yaml not found at {yaml_path}")

## 3️⃣ Train YOLOv5 Model

**Parameters**:
- `--img 640`: Image size
- `--batch 16`: Batch size (reduce to 8 if OOM error)
- `--epochs 100`: Training epochs
- `--data`: Path to data.yaml
- `--weights yolov5n.pt`: Pretrained weights (nano model)
- `--cache`: Cache images for faster training
- `--project`: Output directory
- `--name`: Experiment name

**Estimated Time**: 20-30 minutes on T4 GPU

In [ ]:
# Train the model
print("🚀 Starting training...\n")

!python train.py \
  --img 640 \
  --batch 16 \
  --epochs 100 \
  --data /content/yolov5/dataset/data.yaml \
  --weights yolov5n.pt \
  --cache \
  --project /content/yolov5/runs/train \
  --name emergency_vehicle \
  --exist-ok

print("\n✅ Training completed!")

## 4️⃣ Evaluate Results

In [ ]:
# Display training results
from IPython.display import Image, display

results_path = '/content/yolov5/runs/train/emergency_vehicle'

print("📊 Training Results:\n")

# Results plot
print("📈 Training metrics:")
display(Image(filename=f'{results_path}/results.png', width=800))

# Confusion matrix
print("\n🎯 Confusion Matrix:")
display(Image(filename=f'{results_path}/confusion_matrix.png', width=600))

# Sample predictions
print("\n🖼️ Sample Predictions (Validation Batch):")
display(Image(filename=f'{results_path}/val_batch0_pred.jpg', width=800))

# Print metrics summary
print("\n📋 Final Metrics:")
!cat {results_path}/results.txt | tail -1

## 5️⃣ Export Model (TorchScript)

Export to TorchScript format for deployment

In [ ]:
# Export best model to TorchScript
best_weights = '/content/yolov5/runs/train/emergency_vehicle/weights/best.pt'

if os.path.exists(best_weights):
    print("📦 Exporting model to TorchScript...\n")

    !python export.py \
      --weights {best_weights} \
      --include torchscript \
      --imgsz 640 640

    # Verify export
    torchscript_path = best_weights.replace('.pt', '.torchscript')
    if os.path.exists(torchscript_path):
        file_size = os.path.getsize(torchscript_path) / (1024 * 1024)
        print(f"\n✅ Model exported successfully!")
        print(f"   Location: {torchscript_path}")
        print(f"   Size: {file_size:.2f} MB")
    else:
        print("❌ Export failed")
else:
    print(f"❌ Best weights not found at {best_weights}")

## 6️⃣ Download Results

Download training results and model files

In [ ]:
# Zip results for download
import datetime

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
zip_name = f'emergency_vehicle_results_{timestamp}.zip'

print(f"📦 Creating zip file: {zip_name}")

# Zip the results
!zip -r {zip_name} runs/train/emergency_vehicle

print(f"\n✅ Results zipped: {zip_name}")
print(f"   Size: {os.path.getsize(zip_name) / (1024 * 1024):.2f} MB")

# Download
from google.colab import files
print("\n📥 Downloading...")
files.download(zip_name)

---

## ✅ Training Complete!

**Next Steps**:
1. ✅ Download the results zip file
2. ✅ Extract `best.pt` and `best.torchscript` from weights folder
3. ✅ Copy to your project: `models/best.torchscript`
4. ✅ Test deployment: `python3 src/deployment/main.py`

**Model Location**: `runs/train/emergency_vehicle/weights/best.torchscript`

---

📚 **Documentation**: [YOLOv5 Docs](https://docs.ultralytics.com)

🐛 **Issues**: Check training logs above for errors